**Set environment**

In [1]:
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [4]:
ASSAY=Tewhey_K562_TileMPRA
ls ${FD_RES}/results/${ASSAY}/coverage/track_counts/OL13*hg38* | xargs -n 1 basename

OL13_20220512.hg38.norm.Input.mean.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Input.rep1.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Input.rep2.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Input.rep3.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Input.rep4.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Log2FC.mean.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Output.mean.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Output.rep1.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Output.rep2.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Output.rep3.stranded_pos.bed.gz
OL13_20220512.hg38.norm.Output.rep4.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep1.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep2.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep3.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep4.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep1.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep2.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep3.stranded_pos.bed.gz
OL13_202

In [5]:
ASSAY=Tewhey_K562_TileMPRA
PREFIX=OL13

FD_INP=${FD_RES}/results/${ASSAY}/coverage/track_counts
FP_INPS=($(ls ${FD_INP}/${PREFIX}*hg38*raw*rep*bed.gz))
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_normalized

echo "Total:  ${#FP_INPS[@]}"
for FP_INP in ${FP_INPS[@]}; do
    ###
    FN_INP=$(basename ${FP_INP})
    FN_OUT=${FN_INP}
    FP_OUT=${FD_OUT}/${FN_OUT}
    
    ###
    echo ${FN_INP}
done

Total:  8
OL13_20220512.hg38.raw.Input.rep1.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep2.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep3.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Input.rep4.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep1.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep2.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep3.stranded_pos.bed.gz
OL13_20220512.hg38.raw.Output.rep4.stranded_pos.bed.gz


In [6]:
ASSAY=Tewhey_K562_TileMPRA

FP_LIB=${FD_RES}/results/${ASSAY}/coverage/summary/library_size_summary.tsv
echo ${FP_LIB}
echo 
cat  ${FP_LIB}

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/coverage/summary/library_size_summary.tsv

FName	Size	Prefix	Sample	Group	Type	Genome
OL13_20220512.hg19.norm.Input.rep1.stranded_pos.bed	6418470.212	OL13_20220512	Input.rep1	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep2.stranded_pos.bed	6412310.656	OL13_20220512	Input.rep2	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep3.stranded_pos.bed	6413567.776	OL13_20220512	Input.rep3	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep4.stranded_pos.bed	6410821.564	OL13_20220512	Input.rep4	Input	norm	hg19
OL13_20220512.hg19.norm.Output.rep1.stranded_pos.bed	24214236.952	OL13_20220512	Output.rep1	Output	norm	hg19
OL13_20220512.hg19.norm.Output.rep2.stranded_pos.bed	21967607.311	OL13_20220512	Output.rep2	Output	norm	hg19
OL13_20220512.hg19.norm.Output.rep3.stranded_pos.bed	20773782.111	OL13_20220512	Output.rep3	Output	norm	hg19
OL13_20220512.hg19.norm.Output.rep4.stranded_pos.bed	19239233.544	OL13_20220512	Output.

## RUN

In [7]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 20G \
    --output ${FD_LOG}/coverage_perbase_normalize_subset_TMPRA_OL13.%a.txt \
    --array 0-7 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=Tewhey_K562_TileMPRA
PREFIX=OL13

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### set input and output files
FD_INP=${FD_RES}/results/${ASSAY}/coverage/track_counts
FD_OUT=${FD_RES}/results/${ASSAY}/coverage/track_normalized

FD_LIB=${FD_RES}/results/${ASSAY}/coverage/summary
FN_LIB=library_size_summary.tsv
FP_LIB=${FD_LIB}/${FN_LIB}

FP_INPS=($(ls ${FD_INP}/${PREFIX}*hg38*raw*rep*bed.gz))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})
FN_OUT=${FN_INP}
FP_OUT=${FD_OUT}/${FN_OUT}
PREFIX=${FN_INP%%.gz}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_LIB}
echo
echo "show first few lines of file"
cat ${FP_LIB}
echo
echo ++++++++++++++++++++++++++++++++++++++
echo "Input: " ${FP_INP}
echo
echo "show first few lines of file"
zcat ${FP_INP} | head -5
echo

### execute
STRING1="\t\\."
STRING2="\t0"

zcat ${FP_INP} \
| sed --expression="s/${STRING1}/${STRING2}/g" \
| awk -v PREFIX=${PREFIX} '{print $1 "\t" $2 "\t" $3 "\t" $4 "\t" PREFIX}' \
| join -1 5 -2 1 -t $'\t' -o 1.1,1.2,1.3,1.4,2.2 - ${FP_LIB} \
| awk '{print $1 "\t" $2 "\t" $3 "\t" $4 * 10^6 / $5}' \
| gzip -c \
> ${FP_OUT}

### show I/O file
echo ++++++++++++++++++++++++++++++++++++++
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of file"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29574651


## Check

In [8]:
cat ${FD_LOG}/coverage_perbase_normalize_subset_TMPRA_OL13.0.txt

Hostname:           x3-05-2.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-15-23+13:54:25

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/coverage/summary/library_size_summary.tsv

show first few lines of file
FName	Size	Prefix	Sample	Group	Type	Genome
OL13_20220512.hg19.norm.Input.rep1.stranded_pos.bed	6418470.212	OL13_20220512	Input.rep1	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep2.stranded_pos.bed	6412310.656	OL13_20220512	Input.rep2	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep3.stranded_pos.bed	6413567.776	OL13_20220512	Input.rep3	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep4.stranded_pos.bed	6410821.564	OL13_20220512	Input.rep4	Input	norm	hg19
OL13_20220512.hg19.norm.Output.rep1.stranded_pos.bed	24214236.952	OL13_20220512	Output.rep1	Output	norm	hg19
OL13_20220512.hg19.norm.Output.rep2.stranded_pos.bed	21967607.311	OL13_20220512	Output.rep2	Output	norm	hg19
OL13_20220512.h

In [9]:
cat ${FD_LOG}/coverage_perbase_normalize_subset_TMPRA_OL13.7.txt

Hostname:           x3-03-2.genome.duke.edu
Slurm Array Index:  7
Time Stamp:         03-15-23+13:54:25

++++++++++++++++++++++++++++++++++++++
Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/Tewhey_K562_TileMPRA/coverage/summary/library_size_summary.tsv

show first few lines of file
FName	Size	Prefix	Sample	Group	Type	Genome
OL13_20220512.hg19.norm.Input.rep1.stranded_pos.bed	6418470.212	OL13_20220512	Input.rep1	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep2.stranded_pos.bed	6412310.656	OL13_20220512	Input.rep2	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep3.stranded_pos.bed	6413567.776	OL13_20220512	Input.rep3	Input	norm	hg19
OL13_20220512.hg19.norm.Input.rep4.stranded_pos.bed	6410821.564	OL13_20220512	Input.rep4	Input	norm	hg19
OL13_20220512.hg19.norm.Output.rep1.stranded_pos.bed	24214236.952	OL13_20220512	Output.rep1	Output	norm	hg19
OL13_20220512.hg19.norm.Output.rep2.stranded_pos.bed	21967607.311	OL13_20220512	Output.rep2	Output	norm	hg19
OL13_20220512.h